# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room Part #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
!git clone https://github.com/AI-Maker-Space/DataRepository

Cloning into 'DataRepository'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 68 (delta 18), reused 28 (delta 8), pack-reused 8
Receiving objects: 100% (68/68), 69.00 MiB | 7.42 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "DataRepository/tswift_fued.pdf",
)

documents = loader.load()

In [7]:
documents[0].metadata

{'source': 'DataRepository/tswift_fued.pdf',
 'file_path': 'DataRepository/tswift_fued.pdf',
 'page': 0,
 'total_pages': 22,
 'format': 'PDF 1.4',
 'title': "A Timeline of Taylor Swift and Kim Kardashian's Feud",
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
 'producer': 'Skia/PDF m123',
 'creationDate': "D:20240423220523+00'00'",
 'modDate': "D:20240423220523+00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [9]:
len(documents)

177

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [11]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="Taylor Swift - Fued - ADA",
)

####❓ Question #1:

List out a few of the techniques that FAISS uses that make it performant.

> NOTE: Check the [repository](https://github.com/facebookresearch/faiss) for more information about FAISS!

### Answer:


1.   FAISS is written in C++ and runs everything in memory which makes it fast.
2.   It uses HNSW and NSG to make searching more efficient.
3.   It takes advantage of GPU if available.
4.   It uses quantization to reduce dimensionality and storage.
4.   It has optimized computation of L2 and dot product for vector comparison.



#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [12]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [13]:
retrieved_documents = retriever.invoke("Who is Taylor Swift fueding with?")

In [14]:
for doc in retrieved_documents:
  print(doc)

page_content="4/23/24, 6:05 PM\nA Timeline of Taylor Swift and Kim Kardashian's Feud\nhttps://people.com/taylor-swift-and-kim-kardashian-feud-timeline-8412119\n19/22\nMUSIC" metadata={'source': 'DataRepository/tswift_fued.pdf', 'file_path': 'DataRepository/tswift_fued.pdf', 'page': 18, 'total_pages': 22, 'format': 'PDF 1.4', 'title': "A Timeline of Taylor Swift and Kim Kardashian's Feud", 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m123', 'creationDate': "D:20240423220523+00'00'", 'modDate': "D:20240423220523+00'00'", 'trapped': '', '_id': 'd7ce6b15c504419dbcfc87ac8009890c', '_collection_name': 'Taylor Swift - Fued - ADA'}
page_content="4/23/24, 6:05 PM\nA Timeline of Taylor Swift and Kim Kardashian's Feud\nhttps://people.com/taylor-swift-and-kim-kardashian-feud-timeline-8412119\n21/22\nMUSIC" metadata={'source': 'DataRepository/tswift_fue

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [15]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [16]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [17]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [18]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

### Answer:

The LCEL expression performs the following
1. The question is passed as a context to the retriever using the question. (left side in diagram below)
2. The question is passed directly as a question parameter. (right side in diagram below)
3. Passthrough is necessary to keep the context for the next step.
4. The generator is using the qa prompt
5. The context is generated from the previous step.

Also refer to diagram below...

In [19]:
!pip install -qU grandalf

In [20]:
print(retrieval_augmented_qa_chain.get_graph().draw_ascii())

                       +---------------------------------+                         
                       | Parallel<context,question>Input |                         
                       +---------------------------------+                         
                           *****                   ****                            
                        ***                            ****                        
                     ***                                   ****                    
+--------------------------------+                             **                  
| Lambda(itemgetter('question')) |                              *                  
+--------------------------------+                              *                  
                 *                                              *                  
                 *                                              *                  
                 *                                              *           

Let's test it out!

In [21]:
question = "Who is Taylor Swift fueding with?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Kim Kardashian


In [22]:
question = "Why are they fueding?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content="discussed the longstanding feud between West and Swift in great detail, as\nwell as West's controversial rant during his appearance on Saturday Night", metadata={'source': 'DataRepository/tswift_fued.pdf', 'file_path': 'DataRepository/tswift_fued.pdf', 'page': 6, 'total_pages': 22, 'format': 'PDF 1.4', 'title': "A Timeline of Taylor Swift and Kim Kardashian's Feud", 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m123', 'creationDate': "D:20240423220523+00'00'", 'modDate': "D:20240423220523+00'00'", 'trapped': '', '_id': '8d40e82e48514490a5a20dc56bc6d0b2', '_collection_name': 'Taylor Swift - Fued - ADA'}), Document(page_content='That Beef!" segment. When asked about if her feud with Swift over the\nleaked phone calls was still ongoing, she said she was "over it."', metadata={'source': 'DataRepository/tswif

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room Part #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [23]:
loader = PyMuPDFLoader(
    "DataRepository/tswift_fued.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

### Answer:

To get variety of questions and mitigate biases. This ensures that test data is able to cover a wide range of scenarios.


In [24]:
len(eval_documents)

56

> NOTE: This cell will take ~5-6min. to generate. If you run into any rate-limit issues - please use GPT-3.5-Turbo as your `critic_llm`. If you see any fields marked `nan` - this is product of rate-limiting issues, and you can safely ignore them for now.

In [25]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo") # <--- If you don't have GPT-4 access, or run into rate-limit, or `nan` issues.
# critic_llm = ChatOpenAI(model="gpt-4-turbo")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

/opt/miniconda3/envs/ai-testplanner/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filename and doc_id are the same for all nodes.                   
Generating: 100%|██████████| 20/20 [01:07<00:00,  3.38s/it]


,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How did the singer's experience affect her tru...,[Summer” singer said.\n“You have a fully manuf...,The singer's experience caused her to push awa...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
1,What was the significance of the secretly reco...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",The significance of the secretly recorded phon...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
2,Why was the singer afraid to get on phone calls?,[Summer” singer said.\n“You have a fully manuf...,The singer was afraid to get on phone calls be...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
3,What does the person in the context use their ...,[look in my phone to get a name [of an album]....,The person in the context uses their phone to ...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
4,What was one of the first public interactions ...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...","In 2012, Taylor Swift and Kim Kardashian had o...",simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
5,Who or what are the inspirations for Taylor Sw...,[Who Are Taylor Swift's 'Speak Now' Songs Abou...,nan,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
6,How did the leaked phone call between Taylor S...,"[song. \nDecember 7\n, 2023: Taylor Swift says...",The leaked phone call between Taylor Swift and...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
7,What is the focus of the advertisement for '13...,[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
8,What is the focus of the advertisement for '13...,[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
9,How did Taylor Swift reference taking videos o...,[In the music video for the song’s lead single...,Taylor Swift referenced taking videos on her p...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True


####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

### Answer:

It splits the distribution of type of question. In above, 50% of questions are simple, 25% are reasoning and 25% are multi-context.

Let's look at the output and see what we can learn about it!

In [26]:
testset.test_data[0]

DataRow(question="How did the singer's experience affect her trust in others?", contexts=["Summer” singer said.\n“You have a fully manufactured frame job, in an illegally recorded phone\ncall, which Kim Kardashian edited and then put out to say to everyone that\nI was a liar,” she recalled.\xa0\n“That took me down psychologically to a place I’ve never been before,” the\nGrammy winner continued. “I moved to a foreign country. I didn’t leave a\nrental house for a year. I was afraid to get on phone calls. I pushed away\nmost people in my life because I didn’t trust anyone anymore. I went down\nreally, really hard.”\nAdvertisement\nAd\ni\nThe Rewind: '13 Going 30'\xa0\nSUBSCRIBE\nSKIP TO CONTENT"], ground_truth="The singer's experience caused her to push away most people in her life because she didn't trust anyone anymore.", evolution_type='simple', metadata=[{'source': 'DataRepository/tswift_fued.pdf', 'file_path': 'DataRepository/tswift_fued.pdf', 'page': 15, 'total_pages': 22, 'format':

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [27]:
test_df = testset.to_pandas()

In [28]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How did the singer's experience affect her tru...,[Summer” singer said.\n“You have a fully manuf...,The singer's experience caused her to push awa...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
1,What was the significance of the secretly reco...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",The significance of the secretly recorded phon...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
2,Why was the singer afraid to get on phone calls?,[Summer” singer said.\n“You have a fully manuf...,The singer was afraid to get on phone calls be...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
3,What does the person in the context use their ...,[look in my phone to get a name [of an album]....,The person in the context uses their phone to ...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
4,What was one of the first public interactions ...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...","In 2012, Taylor Swift and Kim Kardashian had o...",simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
5,Who or what are the inspirations for Taylor Sw...,[Who Are Taylor Swift's 'Speak Now' Songs Abou...,nan,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
6,How did the leaked phone call between Taylor S...,"[song. \nDecember 7\n, 2023: Taylor Swift says...",The leaked phone call between Taylor Swift and...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
7,What is the focus of the advertisement for '13...,[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
8,What is the focus of the advertisement for '13...,[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
9,How did Taylor Swift reference taking videos o...,[In the music video for the song’s lead single...,Taylor Swift referenced taking videos on her p...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True


In [29]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [30]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [31]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [32]:
response_dataset[0]

{'question': "How did the singer's experience affect her trust in others?",
 'answer': "The singer's experience made her afraid to get on phone calls and pushed away most people in her life because she didn't trust anyone anymore.",
 'contexts': ['rental house for a year. I was afraid to get on phone calls. I pushed away\nmost people in my life because I didn’t trust anyone anymore. I went down\nreally, really hard.”\nAdvertisement\nAd\ni',
  'Swift said. "I mean, I don\'t think anybody would listen to that and be like,\n\'Oh, that\'s a real diss.\' You gotta tell the story that way that it happened to\nyou and the way you experienced it."',
  'like “a career death” and “took [her] down psychologically to a place\n[she’d] never been before.”\n“Make no mistake — my career was taken away from me,” the “Cruel\nSummer” singer said.',
  'you and the way you experienced it."\nShe went on to add, "If people ask me about it, look, I think it would be\ngreat for me to be like, \'He called me an

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [33]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [34]:
results = evaluate(response_dataset, metrics)

Evaluating: 100%|██████████| 95/95 [00:27<00:00,  3.49it/s]


In [35]:
results

{'faithfulness': 1.0000, 'answer_relevancy': 0.7331, 'context_recall': 0.5702, 'context_precision': 0.5380, 'answer_correctness': 0.5452}

In [36]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How did the singer's experience affect her tru...,The singer's experience made her afraid to get...,[rental house for a year. I was afraid to get ...,The singer's experience caused her to push awa...,1.0,0.886105,1.000000,0.833333,0.991829
1,What was the significance of the secretly reco...,The secretly recorded phone call was significa...,"[never asked to be a part of, since 2009,"" Swi...",The significance of the secretly recorded phon...,1.0,0.907937,0.666667,0.805556,0.737743
2,Why was the singer afraid to get on phone calls?,The singer was afraid to get on phone calls be...,[rental house for a year. I was afraid to get ...,The singer was afraid to get on phone calls be...,1.0,1.000000,1.000000,0.833333,0.527892
3,What does the person in the context use their ...,The person in the context uses their phone to ...,"[Swift continued, ""but maybe all I'll ever get...",The person in the context uses their phone to ...,1.0,0.947564,0.000000,0.250000,0.737931
4,What was one of the first public interactions ...,One of the first public interactions between T...,"[3/22\nNovember 11, 2012: Taylor Swift and Kim...","In 2012, Taylor Swift and Kim Kardashian had o...",1.0,0.902366,1.000000,0.750000,0.738427
5,Who or what are the inspirations for Taylor Sw...,I don't know.,[Who Are Taylor Swift's 'Speak Now' Songs Abou...,nan,NaN,0.000000,0.000000,0.000000,0.198202
6,How did the leaked phone call between Taylor S...,The leaked phone call between Taylor Swift and...,"[March 20, 2020: The full phone call\nbetween ...",The leaked phone call between Taylor Swift and...,1.0,1.000000,1.000000,0.805556,0.615129
7,What is the focus of the advertisement for '13...,I don't know.,[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,NaN,0.000000,0.000000,0.000000,0.198202
8,What is the focus of the advertisement for '13...,I don't know.,[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,NaN,0.000000,0.000000,0.000000,0.198202
9,How did Taylor Swift reference taking videos o...,Taylor Swift referenced taking videos on her p...,[Taylor Swift - Look What You Made Me Do\nTayl...,Taylor Swift referenced taking videos on her p...,1.0,0.949307,1.000000,1.000000,0.744938


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [37]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [39]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [40]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

In [41]:
print(response["answer"])

Taylor Swift is feuding with Kim Kardashian.


In [42]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [45]:
print(response["answer"])

Taylor Swift reclaimed the snake emoji posted in her Instagram comments section by using snake imagery in her sixth studio album Reputation. This appeared to be her way of addressing the controversy and proving that she was not a liar.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [46]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [47]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [48]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 95/95 [00:35<00:00,  2.66it/s]


In [49]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How did the singer's experience affect her tru...,The singer's experience affected her trust in ...,[rental house for a year. I was afraid to get ...,The singer's experience caused her to push awa...,1.00,1.000000,1.000000,0.750000,0.997458
1,What was the significance of the secretly reco...,The secretly recorded phone call between Kanye...,"[never asked to be a part of, since 2009,"" Swi...",The significance of the secretly recorded phon...,1.00,0.922260,1.000000,0.887500,0.669825
2,Why was the singer afraid to get on phone calls?,The singer was afraid to get on phone calls be...,[rental house for a year. I was afraid to get ...,The singer was afraid to get on phone calls be...,1.00,1.000000,1.000000,0.833333,0.603081
3,What does the person in the context use their ...,The person in the context uses their phone to ...,"[Swift continued, ""but maybe all I'll ever get...",The person in the context uses their phone to ...,0.50,0.837838,0.000000,0.200000,0.730004
4,What was one of the first public interactions ...,One of the first public interactions between T...,"[3/22\nNovember 11, 2012: Taylor Swift and Kim...","In 2012, Taylor Swift and Kim Kardashian had o...",1.00,0.944587,1.000000,1.000000,0.738402
5,Who or what are the inspirations for Taylor Sw...,The inspirations for Taylor Swift's 'Speak Now...,[Who Are Taylor Swift's 'Speak Now' Songs Abou...,nan,NaN,0.930692,0.000000,0.000000,0.180410
6,How did the leaked phone call between Taylor S...,The leaked phone call between Taylor Swift and...,"[March 20, 2020: The full phone call\nbetween ...",The leaked phone call between Taylor Swift and...,1.00,0.985841,1.000000,0.873413,0.668579
7,What is the focus of the advertisement for '13...,"The focus of the advertisement is on ""The Rewi...",[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,1.00,0.901252,0.000000,0.000000,0.184411
8,What is the focus of the advertisement for '13...,The focus of the advertisement is on 'The Rewi...,[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,1.00,0.901252,0.000000,0.000000,0.184704
9,How did Taylor Swift reference taking videos o...,"In the music video for ""Look What You Made Me ...",[Taylor Swift - Look What You Made Me Do\nTayl...,Taylor Swift referenced taking videos on her p...,0.75,0.947110,1.000000,1.000000,0.993451


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [51]:
results

{'faithfulness': 1.0000, 'answer_relevancy': 0.7331, 'context_recall': 0.5702, 'context_precision': 0.5380, 'answer_correctness': 0.5452}

And see how our advanced retrieval modified our chain!

In [52]:
advanced_retrieval_results

{'faithfulness': 0.9167, 'answer_relevancy': 0.9191, 'context_recall': 0.5702, 'context_precision': 0.5218, 'answer_correctness': 0.5833}

In [53]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,1.000000,0.916667,-8.333333e-02
1,answer_relevancy,0.733145,0.919059,1.859139e-01
2,context_recall,0.570175,0.570175,1.110223e-16
3,context_precision,0.538012,0.521825,-1.618630e-02
4,answer_correctness,0.545177,0.583347,3.817017e-02


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

Now, lets try using another embedding - embedding-3-small

In [54]:
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Load the documents into qdrant using collection name - "Taylor Swift - Fued - MQR"

In [55]:
vector_store = qdrant = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="Taylor Swift - Fued - MQR",
)

Setup for vector retriever

In [56]:
new_retriever = vector_store.as_retriever()

Setup the retriever using MultiQueryRetriever

In [57]:
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

Setup the retriever chain using the same document chain that was created in earlier.

In [58]:
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

Execute the chain!!

In [59]:
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Combine the results into a dictionary for evaluation of ragas

In [60]:
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Run the evaluation method of ragas

In [61]:
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

Evaluating:  18%|█▊        | 17/95 [00:08<00:43,  1.79it/s]Task exception was never retrieved
future: <Task finished name='Task-1938' coro=<AsyncClient.aclose() done, defined at /opt/miniconda3/envs/ai-testplanner/lib/python3.12/site-packages/httpx/_client.py:2011> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/opt/miniconda3/envs/ai-testplanner/lib/python3.12/site-packages/httpx/_client.py", line 2018, in aclose
    await self._transport.aclose()
  File "/opt/miniconda3/envs/ai-testplanner/lib/python3.12/site-packages/httpx/_transports/default.py", line 385, in aclose
    await self._pool.aclose()
  File "/opt/miniconda3/envs/ai-testplanner/lib/python3.12/site-packages/httpcore/_async/connection_pool.py", line 313, in aclose
    await self._close_connections(closing_connections)
  File "/opt/miniconda3/envs/ai-testplanner/lib/python3.12/site-packages/httpcore/_async/connection_pool.py", line 305, in _close_connections
    await connection.a

Display the results

In [62]:
new_advanced_retrieval_results

{'faithfulness': 0.9537, 'answer_relevancy': 0.8836, 'context_recall': 0.5789, 'context_precision': 0.5291, 'answer_correctness': 0.5908}

Display the result in tabular format along with previous test results

In [63]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,1.000000,0.916667,0.953704,0.037037,-0.046296
1,answer_relevancy,0.733145,0.919059,0.883551,-0.035509,0.150405
2,context_recall,0.570175,0.570175,0.578947,0.008772,0.008772
3,context_precision,0.538012,0.521825,0.529137,0.007312,-0.008874
4,answer_correctness,0.545177,0.583347,0.590759,0.007412,0.045582


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

### Answer:

Yes, based on the data above, text-embedding-3-small improves in terms of answer relevancy.

## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!

We are going to use the latest setup chain using MultiQueryRetriever and text-embedding-3 but send only questions that are multi-context.

In [66]:
# there is only 1 response from synthetic data due to exceeding limit, but lets use this anyway
test_multi_df = test_df[test_df['evolution_type'] == 'multi_context']

In [67]:
test_questions = test_multi_df["question"].values.tolist()
test_groundtruths = test_multi_df["ground_truth"].values.tolist()

answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

results = evaluate(response_dataset, metrics)



I don't know.


Evaluating: 100%|██████████| 35/35 [00:14<00:00,  2.35it/s]


{'faithfulness': 1.0000, 'answer_relevancy': 0.7942, 'context_recall': 0.4524, 'context_precision': 0.5913, 'answer_correctness': 0.5547}


In [69]:
results_multi_df = results.to_pandas()
results_multi_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What part of the speaker's confusion about fli...,I don't know.,[motivation to flip … maybe she took it the wr...,nan,NaN,0.000000,0.000000,0.000000,0.198202
1,What did Kim Kardashian say about Taylor Swift...,Kim Kardashian claimed that Taylor Swift knew ...,"[“Famous” lyric all along \n""She totally appro...",Kim Kardashian claimed that Taylor Swift knew ...,1.0,0.946946,1.000000,1.000000,0.541165
2,What was Taylor Swift's response to Kim Kardas...,Taylor Swift felt like the feud with Kanye Wes...,[19/22\nMUSIC\nTaylor Swift Says Feud with Kan...,A spokesperson for Taylor Swift stated that Ta...,1.0,0.940078,0.000000,0.638889,0.647752
3,What happened with Kim Kardashian and Taylor S...,Kim Kardashian reshared a Taylor Swift song on...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",Kim Kardashian reshares a Taylor Swift song on...,1.0,0.941564,1.000000,0.250000,0.983423
4,What caused the animosity between West and Swi...,The animosity between West and Swift was cause...,[discussed the longstanding feud between West ...,The animosity between West and Swift was cause...,1.0,0.938172,0.000000,1.000000,0.536877
5,What happened between Taylor Swift and Kim Kar...,"At the Grammys, Taylor Swift and Kim Kardashia...","[3/22\nNovember 11, 2012: Taylor Swift and Kim...","At the Grammys, Taylor Swift and Kim Kardashia...",1.0,0.884951,0.500000,0.250000,0.490895
6,"What leaked online on March 20, 2020, about Ka...",The full phone call between Kanye West and Tay...,"[March 20, 2020: The full phone call\nbetween ...",The extended portions of the phone call betwee...,1.0,0.907830,0.666667,1.000000,0.484839
